In [2]:
#KNN____________________________________
# Librerías para manejo de datos
 # librería Natural Language Toolkit, usada para trabajar con textos 
# Librerías para manejo de datos
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
import pandas as pd
pd.set_option('display.max_columns', 25) # Número máximo de columnas a mostrar
pd.set_option('display.max_rows', 50) # Numero máximo de filas a mostar
import numpy as np
np.random.seed(3301)
import pandas as pd
# Para preparar los datos
from sklearn.preprocessing import LabelEncoder
# Para crear el arbol de decisión 
from sklearn.tree import DecisionTreeClassifier 
# Para usar KNN como clasificador
from sklearn.neighbors import KNeighborsClassifier
# Para realizar la separación del conjunto de aprendizaje en entrenamiento y test.
from sklearn.model_selection import train_test_split
# Para evaluar el modelo
from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import plot_confusion_matrix
# Para búsqueda de hiperparámetros
from sklearn.model_selection import GridSearchCV
# Para la validación cruzada
from sklearn.model_selection import KFold 
#Librerías para la visualización
import matplotlib.pyplot as plt
# Seaborn
import seaborn as sns 
from sklearn import tree
pd.options.mode.chained_assignment = None  # default='warn'
import pickle


#Modelo
import sklearn
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

#k fold validation
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold 

#Métricas
from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import plot_confusion_matrix

In [3]:
#Se cargan los datos mediante la libreria pandas y se asigna una variable para los datos
data=pd.read_csv('data-HQ.csv', sep=',', encoding = 'utf-8')
data_preprocesados =data.dropna()
# Ver los datos
display(data_preprocesados.sample(10))

,Unnamed: 0,Unnamed: 0.1,text,class,words
114306,114339,130468,I am getting to the point of killing myself.He...,suicide,get point kil myselfhey everyon nam start feel...
102931,102962,74050,Why cannot everyone stop talking about the nea...,suicide,everyon stop talk near fut try kil myselfnev u...
194647,194708,344258,I just heard someone be transphobic on the rad...,non-suicide,heard someon transphob radio tot angry fuck no...
140241,140287,188653,"At the end of my history class everytime, I tu...",non-suicide,end hist class everytim turn snap potato filt ...
123922,123960,184059,MedicationsWhat medications is anyone else tak...,suicide,medicationswh med anyon els tak cury also sid ...
48650,48660,186023,Dudes be paying for seeing Pussy pics. When th...,non-suicide,dud pay see pussy pic us mir post brought hoe ...
116684,116718,137973,Waking upDoes anyone wake up in the morning an...,suicide,wak updo anyon wak morn disappoint wake updoes...
1172,1172,108811,"everything i have ever wanted, since before i ...",suicide,everyth ev want sint remember know ev going di...
124206,124244,300482,you know that person who no one ever counts on...,non-suicide,know person on ev count anyth everyon know goi...
28698,28704,54642,Story time Like last year I met this really cu...,non-suicide,story tim lik last year met real cut girl lik ...


In [4]:
# Dimensiones de los datos
data_preprocesados.shape

(195639, 5)

In [5]:
display(data_preprocesados.head(5)) # Primeras Filas
# Imprimimos los diferentes tipos de las columnas
data_preprocesados.dtypes

,Unnamed: 0,Unnamed: 0.1,text,class,words
0,0,173271,i want to destroy myselffor once everything wa...,suicide,want destroy myselff everyth start feel okay c...
1,1,336321,I kind of got behind schedule with learning fo...,non-suicide,kind got behind schedule learn next week testw...
2,2,256637,I am just not sure anymoreFirst and foremost: ...,suicide,sur anymorefirst foremost brazil judg second n...
3,3,303772,please give me a reason to liveThats too much ...,suicide,pleas giv reason liveth much reason liv lik an...
4,4,293747,27f struggling to find meaning moving forwardI...,suicide,27f struggling find mean mov forward admit bit...


Unnamed: 0       int64
Unnamed: 0.1     int64
text            object
class           object
words           object
dtype: object

In [6]:
#Separar datos
X_data, y_data = data_preprocesados['words'],data_preprocesados['class']
y_data = (y_data == 'suicide').astype(int)
y_data

0         1
1         0
2         1
3         1
4         1
         ..
195634    0
195635    0
195636    0
195637    0
195638    1
Name: class, Length: 195639, dtype: int64

In [7]:
# Primero, es fundamental saber que un vector es una "recopilación de celdas o casilla que contienen variables individuales"
# En este paso, se vectorizan los datos por conteo de palabras. 
count = CountVectorizer()
X_count = count.fit_transform(X_data)
# Se revisa que se siga teniendo el mismo numero de datos que se tenia originalmente. 
print(X_count.shape)

(195639, 204752)


In [8]:
#KNN____________________________________________

# Se borran las columnas de unnamed: 0 y Unnames: 0.1, ya que no afecta el algoritmo y no son relevantes en este caso. 

data_borrandoColumnas = data_preprocesados
data_borrandoColumnas = data_borrandoColumnas.drop(['Unnamed: 0'], axis=1)
data_borrandoColumnas = data_borrandoColumnas.drop(['Unnamed: 0.1'], axis=1)
display(data_borrandoColumnas.head(5))

,text,class,words
0,i want to destroy myselffor once everything wa...,suicide,want destroy myselff everyth start feel okay c...
1,I kind of got behind schedule with learning fo...,non-suicide,kind got behind schedule learn next week testw...
2,I am just not sure anymoreFirst and foremost: ...,suicide,sur anymorefirst foremost brazil judg second n...
3,please give me a reason to liveThats too much ...,suicide,pleas giv reason liveth much reason liv lik an...
4,27f struggling to find meaning moving forwardI...,suicide,27f struggling find mean mov forward admit bit...


Teniendo en cuenta que el objetivo es a partir de textos planos, si una persona está intentando, va a intentar o ha intentado suicidarse con el fin de prevenir 
estos acontecimientos y/o brindar la ayuda necesaria a la gente que lo necesita, entonces se puede concluir que la variable objetivo es la "class", la cual dice si es suicide o non-suicide el 
texto 

In [9]:
Y = data_borrandoColumnas['class']
X =data_borrandoColumnas.drop(['class'], axis=1)

In [10]:
# Al ya haber escogido la variable objetivo, ya se puede dividir los datos en entrenamiento y test
X_train, X_test, Y_train, Y_test = train_test_split(X_count, y_data, test_size=0.2, random_state=0)
# se muestra
display(X.head(5)) # Primeras Filas

,text,words
0,i want to destroy myselffor once everything wa...,want destroy myselff everyth start feel okay c...
1,I kind of got behind schedule with learning fo...,kind got behind schedule learn next week testw...
2,I am just not sure anymoreFirst and foremost: ...,sur anymorefirst foremost brazil judg second n...
3,please give me a reason to liveThats too much ...,pleas giv reason liveth much reason liv lik an...
4,27f struggling to find meaning moving forwardI...,27f struggling find mean mov forward admit bit...


In [ ]:
arrayTrain = X_train.toarray()
arrayPredict = X_test.toarray()
# Imprime los lengths del array. 
print(len(arrayTrain))
print(len(arrayPredict))

In [ ]:
neigh = KNeighborsClassifier(n_neighbors=3)
neigh = neigh.fit(X_train, Y_train)
y_pred = neigh.predict(X_test)

# Se genera la matriz de confusión
confusion_matrix(Y_test, y_pred)

# Mostrar reporte de clasificación
print(classification_report(Y_test, y_pred))

In [ ]:
print('Exactitud: %.2f' % accuracy_score(Y_test, y_pred))
print("Recall: {}".format(recall_score(Y_test,y_pred)))
print("Precisión: {}".format(precision_score(Y_test,y_pred)))
print("Puntuación F1: {}".format(f1_score(Y_test,y_pred)))